# Języki symboliczne - rok akademicki 2021/2022

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`

Przed wysłaniem notatnika **upewnij się jeszcze raz** że zmieniłeś nazwę i że rozwiązałeś wszystkie zadania/ćwiczenia, w szczególności, że uzupełniłeś wszystkie pola `YOUR CODE HERE` oraz `YOUR ANSWER HERE`.

# Temat: Klasy - iteratory, generatory, wyrażenia generatora.
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.


## Iteratory

https://docs.python.org/3/tutorial/classes.html#Iterators

- po większości obiektów będących kontenerami możemy iterować używając pętli `for`;
- funkcjonalność ta realizowana jest poprzez wywołanie przez `for` metody `iter()` na obiekcie kontenera, która zwraca obiekt iteratora definiujący metodę `__next__()` udostępniającą raz po razie kolejne elementy kontenera;
- w przypadku gdy nie ma już kolejnych elementów `__next__()` zgłasza wyjątek `StopIteration` powiadamiajacy pętlę `for`, że należy ją zakończyć;
- `__next__()` może być wywołane poprzez funkcję wbudowaną `next()`;



In [1]:
for element in 'abcd':
    print(element, end = ' ')

a b c d 

In [2]:
s = 'abcd'
it = iter(s)     # wywołanie metody iter() na obiekcie kontenera (str) zwraca obiekt iteratora definujący metodę __next__()
print(it)
print(next(it))  # wywołuje metodę __next__() 
print(next(it))
print(next(it))
print(next(it))
next(it)          # zgłoszenie wyjątku StopIteration 

a
b
c
d


StopIteration: 

Znając mechanizmy iteratora stosunkowo łatwo jest go zaimplementować w swoich klasach.

- należy zdefiniować metodę `__iter__()` zwracającą obiekt z metodą `__next__()`. 
- Jeśli klasa definiuje `__next__()` to `__iter__()` zwraca po prostu `self`.

__Przykład__. Napisz klasę`Wspak` przyjmującą jako argument konstruktora tekst. Klasa ta powinna pełnić rolę iteratora, iterującego po podanym tekście wspak.

In [3]:
class Wspak:
    def __init__(self, text):
        self.text = text
        self.index = len(text)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.text[self.index]

In [4]:
wsp = Wspak('spam') # konkretyzacja klasy  - iterator
print(wsp)
print(type(wsp))
for znak in wsp:
    print(znak)

<class '__main__.Wspak'>
m
a
p
s


In [5]:
next(wsp) # zgłasza wyjątek StopIteration 

StopIteration: 

In [6]:
wsp1 = Wspak('maps')
wsp1.__next__(), wsp1.__next__(), wsp1.__next__(), wsp1.__next__() # wywołanie metody __next__()
#wsp1.__next__() # to wywołanie metody __next__() zgłasza wyjątek StopIteration 

('s', 'p', 'a', 'm')

## Generatory

https://docs.python.org/3/tutorial/classes.html#generators

W Pythonie mamy dwa rodzaje generatorów:
- tworzone przez funkcje generujące;
- tworzone przez wyrażenia generujące.


Generatory można traktować jak funkcje, których działanie można wstrzymywać i wznawiać (zmienne lokalne nie są niszczone po opuszczeniu generatora umożliwiając późniejsze wznowienie w tym samym miejscu):
- Na potrzeby generatorów zostało wprowadzone nowe słowo kluczowe `yield`. 
- Każda funkcja, która zawiera instrukcję `yield` staje się automatycznie funkcją generującą. 
- Przy wywołaniu funkcji generującej nie jest zwracana pojedyncza wartość. Zamiast tego zwracany jest obiekt generatora, który obsługuje protokół iteratora.
- instrukcja `yield` występuje zamiast instrukcji `return`.  Różnica pomiędzy tymi instrukcjami polega jednak na tym, że w przypadku instrukcji `yield` zostaje zapamiętany stan wykonywania generatora oraz wartości wszystkich zmiennych lokalnych. Przy kolejnym wywołaniu metody `__next()__` generatora wykonywanie funkcji jest wznawiane bezpośrednio po ostatnio napotkanej instrukcji `yield`.




In [7]:
def wygeneruj_calkowite(N):
    for i in range(N):
        yield i              # yield oznacza, że jest to generator - nie zwykła funkcja

g = wygeneruj_calkowite(10); # funkcja wygeneruj_calkowite(10) zwraca obiekt generatora
print(g)

for j in g:
    print(j, end = ' ')

<generator object wygeneruj_calkowite at 0x000001E5DEF75270>
0 1 2 3 4 5 6 7 8 9 

- Generator (funkcja realizująca generator) generuje i zwraca za każdym razem (z każdym wywołaniem funkcji) jedną wartość.
- Wywołanie po raz pierwszy metody generatora `__next__()` dla obiektu generatora powoduje wykonanie kodu w `utworz_licznik` do pierwszego wystąpienia `yield`, a następnie zwrócenie wydobytej wartości (w przykładzie poniżej 10).
- Kolejne wywołania `__next__()` rozpoczynają od miejsca ostatniego wyjścia z funkcji i działają do kolejnego napotkania `yield`. 

In [8]:
def utworz_licznik(x):
    print('Wejście do licznika')
    while True:
        yield x                   # yield oznacza, że jest to generator
        print('zwiększenie x')
        x = x + 1

l = utworz_licznik(10)            # funkcja utworz_licznik(10) zwraca obiekt generatora
print(l)

next(l), next(l), next(l)

<generator object utworz_licznik at 0x000001E5DEF750B0>
Wejście do licznika
zwiększenie x
zwiększenie x


(10, 11, 12)

- wszystko co może być zrobione za pomocą generatorów można zaimplementować również za pomocą iteratorów bazujących na klasach.
- to co wpływa na kompaktowość generatorów to to, że metody `__iter__()` oraz `__next__()` są tworzone automatycznie
- kolejną, kluczową ich cechą jest pamiętanie lokalnych zmiennych i stanu wykonywanego kodu pomiędzy kolejnymi do niego odwołaniami.
- to upraszcza implementację w stosunku do podejścia w którym używa się `self.index` i `self.data`.
- w przypadku zakończenia działania generatora zgłasza on automatycznie wyjątek `StopIteration`.

In [9]:
# generator
'''
def utworz(N):
    for i in range(N):
        yield i 
'''
# iterator
class Utworz_1:  
    def __init__(self, data):
        self.data = data
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == self.data:
            raise StopIteration
        self.index = self.index + 1
        return self.index

In [10]:
gg = Utworz_1(10); # konkretyzacja klasy  - iterator
print(gg)

for j in gg:
    print(j, end = ' ')

1 2 3 4 5 6 7 8 9 10 

## Wyrażenia generujące (`generator expressions`)

https://docs.python.org/3/tutorial/classes.html#generator-expressions


- Niektóre proste generatory mogą być efektywnie implementowane jako wyrażenia przy pomocy mechanizmu typu `list comprehension`, ale zamiast nawiasów kwadratowych `[]` używają nawiasów okrągłych `()`.

- Wyrażenia te przeznaczone są dla sytuacji w których generator jest użyty bezpośrednio w wywołującej go funkcji.


In [11]:
g = (i**2 for i in range(10))
print(g)
for j in g:
    print(j, end = ' ')

<generator object <genexpr> at 0x000001E5DEF90D60>
0 1 4 9 16 25 36 49 64 81 

Wyrażenia generatora często są używane z takimi funkcjami jak `sum`, `min`, `max`.

In [12]:
sum(i**2 for i in range(10)) # użycie generatora bezpośrednio w funkcji sum()

285

## Przestrzenie nazw. Polecenia `nonlocal`, `global`.

- Przestrzenie nazw tworzone są w różnych chwilach i są aktywne przez różny czas.


- Przestrzeń nazw zawierająca nazwy wbudowane tworzona jest podczas rozpoczęcia pracy interpretera Pythona i nigdy nie jest usuwana. Nazwy wbudowane przechowywane są w module o nazwie `builtins`.


- Przestrzeń nazw globalnych modułu tworzona jest podczas wczytywania jego definicji i jest aktywna również do chwili zakończenia pracy interpretera.


- Instrukcje wykonywane przez wywołania interpretera, zarówno czytane z pliku jak i wprowadzane interaktywnie, są częścią modułu o nazwie `__main__` - tak więc posiadają swoją własną przestrzeń nazw globalnych.


- Przestrzeń nazw lokalnych funkcji tworzona jest w momencie jej wywołania i niszczona, gdy następuje powrót z funkcji lub zgłoszony został w niej wyjątek, który nie został tam obsłużony.


- Wywołanie rekurencyjne powoduje tworzenie za każdym razem nowej przestrzeni nazw lokalnych.


- W każdym momencie wykonania programu, istnieją co najmniej trzy zagnieżdżone zasięgi nazw (tzn. wprost osiągalne są trzy przestrzenie nazw):
  - najbardziej zagnieżdżony, w którym najpierw poszukuje się nazwy, zawiera on nazwy lokalne; 
  - środkowy, przeszukiwany w następnej kolejności, który zawiera aktualne nazwy globalne modułu; 
  - zewnętrzny (przeszukiwany na końcu) jest zasięgiem nazw wbudowanych.


__`globals()`__ -  zwraca słownik reprezentujący bieżącą globalną tablicę symboli. Jest to zawsze słownik bieżącego modułu (wewnątrz funkcji lub metody jest to moduł, w którym jest zdefiniowany, a nie moduł, z którego jest wywoływany).

__`locals()`__ - zwraca słownik reprezentujący bieżącą lokalną tablicę symboli. Na poziomie modułu `locals()` i `globals()` są tym samym słownikiem.



In [13]:
# Przed każdym uruchomieniem naciśnij: Kernel / Restart & clear output

def scope_test():
    
    def do_local():
        spam = "test_3"
        print('3)\n', locals())

    spam = "test_2"
    print('2)\n', locals())
    do_local()

spam = 'test_1'    
print('1)\n', locals())
scope_test()

locals() == globals()

#dir(__builtins__) # zwraca nazwy zdefiniowane w module builtins

1)
 {'__name__': '__main__', '__doc__': '\ndef utworz(N):\n    for i in range(N):\n        yield i \n', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', "for element in 'abcd':\n    print(element, end = ' ')", "s = 'abcd'\nit = iter(s)     # wywołanie metody iter() na obiekcie kontenera (str) zwraca obiekt iteratora definujący metodę __next__()\nprint(it)\nprint(next(it))  # wywołuje metodę __next__() \nprint(next(it))\nprint(next(it))\nprint(next(it))\nnext(it)          # zgłoszenie wyjątku StopIteration ", 'class Wspak:\n    def __init__(self, text):\n        self.text = text\n        self.index = len(text)\n\n    def __iter__(self):\n        return self\n\n    def __next__(self):\n        if self.index == 0:\n            raise StopIteration\n        self.index = self.index - 1\n        return self.text[self.index]', "wsp = Wspak('spam') # konkretyzacja klasy  - iterat

True

### Deklaracje `nonlocal`, `global`.

- deklaracji `nonlocal` używamy aby odwołać się do zmiennych znajdujących się poza najbardziej zagnieżdżonym zasięgiem. Bez niej zmienne te byłyby tylko do odczytu. Podczas zapisu tworzona by była zmienna o tej samej nazwie, a ta zewnętrzna pozostała by niezmieniona.


- deklaracja `global` może zostać użyta do oznaczenia, że wyszczególniona nazwa należy do przestrzeni nazw globalnych


- Zasięgi nazw zdeterminowane są przez ich zasięg w tekście. Zasięg globalny funkcji zdefiniowanej w danym module jest zasięgiem związanym z tym modułem, niezależnie gdzie i jak (przy pomocy jakiego aliasu funkcja jest wywoływana)


- Jedną z cech szczególnych Pythona jest to, że przypisanie zawsze zachodzi w najbardziej zagnieżdżonym zasięgu. Przypisania nie powodują kopiowania danych - przywiązują jedynie nazwy do obiektów.


- To samo zachodzi w przypadku usuwania: instrukcja `del x` usuwa związek obiektu identyfikowanego przez nazwę x z tą nazwą w przestrzeni nazw lokalnych

In [14]:
def scope_test():
    def do_local():
        spam = "local spam"     # lokalne związanie nie zmienia spam w scope_test()

    def do_nonlocal():
        nonlocal spam           # nonlocal zmienia spam w scope_test()
        spam = "nonlocal spam"  

    def do_global():
        global spam             # global zmienia spam na poziomie modułu
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)
    
scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


## Zadanie 1

Napisz klasę`SamogloskaUpss` przyjmującą jako argument konstruktora tekst. Klasa ta powinna pełnić rolę iteratora, iterującego po podanym tekście. W przypadku napotkania samogłoski iterator zwraca tekst `Upss`.

https://docs.python.org/3/tutorial/classes.html#iterators

In [35]:
# YOUR CODE HERE
#raise NotImplementedError()

class SamogloskaUpss:
    def __init__(self, text):
        self.text = text
        self.index = - 1
        self.samogloski = "aąeęouiy"

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == len(self.text) - 1:
            raise StopIteration
        self.index = self.index + 1
        if self.text[self.index].lower() in self.samogloski:
            return self.text[self.index] + " - Upss"
        else:
            return self.text[self.index]


sam = SamogloskaUpss('spamSPAM')
for znak in sam:
    print(znak)

s
p
a - Upss
m
S
P
A - Upss
M


## Zadanie 2
Napisz klasę `Unikalne` przyjmującą jako argument konstruktora pewną listę. Klasa ta powinna pełnić rolę iteratora, iterującego po unikalnych (występujących dokładnie jeden raz) elementach listy.


In [82]:
# YOUR CODE HERE
#raise NotImplementedError()

class Unikalne:
    def __init__(self, lista1 : list):
        self.lista = lista1
        self.index = - 1
        self.unik = []

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == len(self.lista) - 1:
            raise StopIteration
        self.index = self.index + 1
        while self.lista[self.index] in self.unik:
            self.index = self.index + 1
            if self.index == len(self.lista):
                raise StopIteration
        else:
            self.unik.append(self.lista[self.index])
            return self.lista[self.index]



sam = Unikalne(["a", "b", "c", "a", "t", "T","a","a", "t", "S", "S","S", "a", "T", "z", "r", "r"])
for znak in sam:
    print(znak)

a
b
c
t
T
S
z
r


## Zadanie 3
Napisz funkcję `liczby_pierwsze` przyjmującą jako argument liczbę całkowitą. Funkcja ta powinna być generatorem zwracającym liczby pierwsze od 2 do liczby przesłanej jako argument, bez niej. Wykorzystaj słowo kluczowe `yield`.

Przykładowe użycie funkcji `liczby_pierwsze`:


```python
for i in liczby_pierwsze(100):
    print(i)
# powyższy kod powinien wypisać liczby pierwsze do liczby 97 włącznie
```

https://docs.python.org/3/tutorial/classes.html#generators

In [83]:
# YOUR CODE HERE
#raise NotImplementedError()

def liczby_pierwsze(stop):
    for x in range(2, stop):
        for i in range(2, x):
            if x % i == 0:
                break
        else:
            yield x


for i in liczby_pierwsze(12):
    print(i)

2
3
5
7
11


## Zadanie 4
Napisz funkcję `JednorekiBandyta`, przyjmującą jako argument pewną kwotę w groszach. Funkcja ta powinna być generatorem tworzącym obiekty monet o nominałach 1 lub 2 grosze, do kwoty zadanej argumentem. Klasa `Moneta` z poprzednich laboratoriów.

Przykładowo, wywołanie `JednorekiBandyta(100)` powinno wygenerować na przykład 100 monet 1gr, lub 50 monet 2gr, lub dowolną inną kombinację dającą w sumie 100gr.


In [160]:
# YOUR CODE HERE
#raise NotImplementedError()
from decimal import *
getcontext().prec = 16
class Moneta():
    def __init__(self, sWartosc, sWaluta = "PLN"):
        self._waluta = sWaluta
        if sWartosc in [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]:
            self._wartosc = Decimal(str(sWartosc))
        else:
            self._wartosc = 0

    def pobierz_wartosc(self):
        return self._wartosc

    def pobierz_walute(self):
        return self._waluta

    def __str__(self):
        return f'Moneta {self._wartosc} {self._waluta}'

    def __repr__(self):
        return f"Moneta({self._wartosc}, '{self._waluta}')"

In [161]:
def JednorekiBandytwa(kwota):
    for i in range(1, int(kwota // 2 + 1)):
        yield Moneta(0.02)
    if kwota % 2 != 0:
        yield Moneta(0.01)



for i in JednorekiBandytwa(11):
    print(i.pobierz_wartosc())

0.02
0.02
0.02
0.02
0.02
0.01


## Zadanie 5

Do klasy `PrzechowywaczMonet` z poprzednich laboratoriów dodaj metodę `wszystkieMonety` będącą generatorem zwracającym kolejno wszystkie przechowywane monety posortowane rosnąco po nominale.


In [166]:
# YOUR CODE HERE
#raise NotImplementedError()
class PrzechowywaczMonet:
    def __init__(self, lis = ["PLN"]):
        self._lista = []
        self._waluta = lis


    def wszystkieMonety(self):
        self._lista.sort(key = lambda x: x._wartosc)
        for i in range(0, len(self._lista)):
            yield self._lista[i]


    def dodaj_monete(self, moneta : Moneta):
        if isinstance(moneta, Moneta):
            if moneta.pobierz_walute() in self._waluta:
                self._lista.append(moneta)
            else:
                print("Nieznana moneta")
        else:
            print("Przeslany obiekt nie jest moneta")


    def suma(self):
        suma = 0
        for _ in self._lista:
            suma += _.pobierz_wartosc()
        return suma


    def zwroc_monete(self, x):
        for _ in self._lista:
            if _.pobierz_wartosc() == x:
                self._lista.remove(_)
                return _
        else:
            print("Nie ma takiej monety")

In [167]:
m001 = Moneta(0.01)
m002 = Moneta(0.02)
m005 = Moneta(0.05)
m02 =  Moneta(0.2)
m01 =  Moneta(0.1)
przechowywacz = PrzechowywaczMonet()
przechowywacz.dodaj_monete(m001)
przechowywacz.dodaj_monete(m002)
przechowywacz.dodaj_monete(m005)
przechowywacz.dodaj_monete(m02)
przechowywacz.dodaj_monete(m01)

for i in przechowywacz.wszystkieMonety():
    print(i.__str__())

Moneta 0.01 PLN
Moneta 0.02 PLN
Moneta 0.05 PLN
Moneta 0.1 PLN
Moneta 0.2 PLN


## Zadanie 6
Napisz funkcję liczącą sumę iloczynów elementów dwóch list `a` i `b`: 

`a[0]*b[0]+a[1]*b[1]+....`

Obie listy powinny być równej długości. Wykorzystaj `generator expressions` oraz funkcję `zip`.

https://docs.python.org/3/tutorial/classes.html#generator-expressions

In [168]:
# YOUR CODE HERE
#raise NotImplementedError()
def zad_6(a : list, b : list):
    return sum(x * y for x, y in zip(a, b))

l_a = [1, 2, 3, 4]
l_b = [5, 6, 7, 8]
print(zad_6(l_a, l_b))

70


## Zadanie 7

Napisz funkcję `srednia` będącą generatorem, liczącym średnią arytmetyczną wartości przesłanych do niej przy pomocy metody `send`.

Przykładowy kod używający `send`:

```python
def sumowanie():
    suma=0
    while True:
        a=(yield)
        print("Otrzymano:", a)
        suma+=a
        yield suma

generator=sumowanie()

for i in [1, 2, 3, 5, 7, 11]:
    next(generator)
    print("Generator zwrócił:", generator.send(i))
```



https://docs.python.org/3/reference/expressions.html#yield-expressions

In [183]:
# YOUR CODE HERE
#raise NotImplementedError()
def srednia():
    suma=0
    licznik = 1
    lista = []
    while True:
        a = (yield)
        lista.append(a)
        print("Srednia z", lista, "= ", end="")
        suma += a
        srednia = suma / licznik
        licznik += 1
        yield srednia

generator=srednia()

for i in [1, 2, 3, 5, 7, 11]:
    next(generator)
    print(generator.send(i))

Srednia z [1] = 1.0
Srednia z [1, 2] = 1.5
Srednia z [1, 2, 3] = 2.0
Srednia z [1, 2, 3, 5] = 2.75
Srednia z [1, 2, 3, 5, 7] = 3.6
Srednia z [1, 2, 3, 5, 7, 11] = 4.833333333333333


## Zadanie 8

Dany jest kod:

```python
x=0

def bla():
    x+=1
    
print(x)
bla()
print(x)
```

Zmodyfikuj funkcję `bla` tak, aby zwiększała wartość zmiennej globalnej `x` o 1.

Wykorzystaj słowo kluczowe `global`.



In [198]:
# YOUR CODE HERE
#raise NotImplementedError()
x=0

def bla():
    global x
    x+=1

print(x)
bla()
print(x)

0
1


## Zadanie 9

Dany jest kod:

```python
x=1337

def foo():
    def bar():
        x+=1
    x=0
    print(x)
    bar()
    print(x)
    
print(x) 
foo()
print(x)

```

Zmodyfikuj funkcje `foo` i `bar` tak, aby funkcja `bar` zwiększała zmienną `x` zdefiniowaną w funkcji `foo` o 1, nie zmieniając wartości zmiennej globalnej `x`. 

Wykorzystaj słowo kluczowe `nonlocal`.

In [201]:
# YOUR CODE HERE
#raise NotImplementedError()
x=1337

def foo():
    def bar():
        nonlocal x
        x+=1
    x=0
    print(x)
    bar()
    print(x)

print(x)
foo()
print(x)

1337
0
1
1337
